In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset from CSV (Replace 'hotel_data.csv' with your actual file path)
data = pd.read_csv('/content/hotel_price_dataset_weekend.csv')

# Display the first few rows to understand its structure
print(data.head())







FileNotFoundError: [Errno 2] No such file or directory: '/content/hotel_price_dataset_weekend.csv'

In [ ]:
# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])  # Or use the correct format: format='%Y-%m-%d'

# Extract Day and Month from Date
data['Day'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month

# Extract Weekday or Weekend (0 = Monday, 6 = Sunday)
data['Weekday'] = data['Date'].dt.weekday
data['Is_Weekend'] = data['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

# Use the existing 'Holiday' column directly (encoded already in the dataset)
data['Is_Holiday'] = data['Holiday'].astype(int)

# Label Encoding for Room_Type (convert "Normal", "Deluxe", "Super Deluxe" to 0, 1, 2)
label_encoder = LabelEncoder()
data['Room_Type'] = label_encoder.fit_transform(data['Room_Type'])

# Check the transformed dataset
print(data[['Booking_ID', 'Room_Type', 'Date', 'Is_Holiday', 'Is_Weekend', 'Day', 'Month', 'Price']].head())

In [ ]:
X = data[['Day', 'Month', 'Is_Weekend', 'Is_Holiday', 'Room_Type']]  # Room_Type is now encoded as a single column
y = data['Price']  # Use the provided 'Price' column directly

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features for the DNN model (important for neural networks)
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Mean Squared Error for Random Forest: {mse_rf}')

In [ ]:
dnn_model = Sequential()
dnn_model.add(Dense(64, input_dim=X_scaled_train.shape[1], activation='relu'))
dnn_model.add(Dense(32, activation='relu'))
dnn_model.add(Dense(1))  # Output layer with one unit (price)

# Compile the model
dnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the DNN model
dnn_model.fit(X_scaled_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the DNN model
loss_dnn = dnn_model.evaluate(X_scaled_test, y_test)
print(f'Mean Squared Error for DNN: {loss_dnn}')

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 10260548.0000 - val_loss: 10060511.0000
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10430872.0000 - val_loss: 10049996.0000
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10377152.0000 - val_loss: 10032233.0000
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10541632.0000 - val_loss: 10004069.0000
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10587025.0000 - val_loss: 9961847.0000
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10364026.0000 - val_loss: 9900636.0000
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10488405.0000 - val_loss: 9815383.0000
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10428142.0000 - val_loss: 9701921.0000
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10154369.0000 - val_loss: 9555516.0000
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9931657.0000 - val_loss: 9372575.0000
Epoch 11/100
22/22 ━━━━━━━━

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv("/content/hotel_price_dataset_weekend.csv")

# Preprocessing: Convert 'Date' to datetime format and extract day and month
# Convert 'Date' column to datetime format (automatically inferring the format)
data['Date'] = pd.to_datetime(data['Date'])

# Extract Day and Month from Date
data['Day'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month


# Encoding Room_Type (assuming you have 3 categories: Normal, Deluxe, Super Deluxe)
le = LabelEncoder()
data['Room_Type_Encoded'] = le.fit_transform(data['Room_Type'])
data['Is_Holiday'] = data['Holiday'].astype(int)
data['Is_Weekend'] = data['Date'].dt.weekday >= 5
# Feature Selection
X = data[['Day', 'Month', 'Is_Weekend', 'Is_Holiday', 'Room_Type_Encoded']]  # Include relevant features
y = data['Price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features (necessary for models like XGBoost or Neural Networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model 1: XGBoost
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=10, subsample=0.8, colsample_bytree=0.8)
xgb_model.fit(X_train_scaled, y_train)

# Model 2: Random Forest (with hyperparameter tuning)
rf_model = RandomForestRegressor(n_estimators=300, max_depth=15, min_samples_split=10, min_samples_leaf=1, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Model 3: Deep Neural Network (DNN)
dnn_model = Sequential()
dnn_model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
dnn_model.add(Dropout(0.3))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dropout(0.3))
dnn_model.add(Dense(32, activation='relu'))
dnn_model.add(Dense(1))  # Output layer (Price prediction)

# Compile the DNN model
dnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the DNN model
dnn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)

# Predictions for each model
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_rf = rf_model.predict(X_test_scaled)
y_pred_dnn = dnn_model.predict(X_test_scaled)

# Evaluate each model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_dnn = mean_squared_error(y_test, y_pred_dnn)
r2_xgb = r2_score(y_test, y_pred_xgb)
r2_rf = r2_score(y_test, y_pred_rf)
r2_dnn = r2_score(y_test, y_pred_dnn)

print(f"XGBoost MSE: {mse_xgb}, R2: {r2_xgb}")
print(f"Random Forest MSE: {mse_rf}, R2: {r2_rf}")
print(f"DNN MSE: {mse_dnn}, R2: {r2_dnn}")

# Ensemble Model (Voting Regressor)
ensemble_model = VotingRegressor(estimators=[('rf', rf_model), ('xgb', xgb_model)])
ensemble_model.fit(X_train_scaled, y_train)

# Predictions for the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test_scaled)
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)

print(f"Ensemble Model MSE: {mse_ensemble}, R2: {r2_ensemble}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 10667478.0000
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10567978.0000
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10275934.0000
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9542605.0000
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7324641.5000
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4480187.0000
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1553577.5000
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 559333.9375
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 572953.5625
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 489784.5312
Epoch 11/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 459687.5938
Epoch 12/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 448723.2188
Epoch 13/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 441458.0000
Epoch 14/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss

In [ ]:
import datetime
import numpy as np

def get_user_input():
    # Get user input
    date_str = input("Enter Date (YYYY-MM-DD): ")
    is_holiday = int(input("Is it a holiday? (1 for Yes, 0 for No): "))
    room_type = input("Enter Room Type (Normal, Deluxe, Super Deluxe): ")

    # Convert date string to datetime object
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d")

    # Extract day and month
    day = date.day
    month = date.month

    # Determine if it's a weekend
    is_weekend = 1 if date.weekday() >= 5 else 0

    # Encode Room Type
    room_type_mapping = {"Normal": 0, "Deluxe": 1, "Super Deluxe": 2}
    room_type_encoded = room_type_mapping.get(room_type, -1)

    if room_type_encoded == -1:
        print("Invalid Room Type. Please enter one of the following: Normal, Deluxe, Super Deluxe.")
        return None

    return np.array([[day, month, is_weekend, is_holiday, room_type_encoded]])

def predict_price(user_input):
    if user_input is None:
        return

    # Scale the input features
    user_input_scaled = scaler.transform(user_input)

    # Get predictions
    price_xgb = xgb_model.predict(user_input_scaled)[0]
    price_rf = rf_model.predict(user_input_scaled)[0]
    price_dnn = dnn_model.predict(user_input_scaled)[0][0]  # Extracting single value from DNN output
    price_ensemble = ensemble_model.predict(user_input_scaled)[0]

    # Display results
    print("\nPredicted Prices:")
    print(f"XGBoost Model Price: ₹{price_xgb:.2f}")
    print(f"Random Forest Model Price: ₹{price_rf:.2f}")
    print(f"Deep Neural Network Model Price: ₹{price_dnn:.2f}")
    print(f"Ensemble Model Price: ₹{price_ensemble:.2f}")

# Get user input and predict price
user_input = get_user_input()
predict_price(user_input)

Enter Date (YYYY-MM-DD): 2024-04-14
Is it a holiday? (1 for Yes, 0 for No): 0
Enter Room Type (Normal, Deluxe, Super Deluxe): Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

Predicted Prices:
XGBoost Model Price: ₹2894.34
Random Forest Model Price: ₹2895.04
Deep Neural Network Model Price: ₹3004.15
Ensemble Model Price: ₹2894.69


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
